In [1]:
import pandas as pd
from pymongo import MongoClient
from IPython.display import display  # 用於在 Notebook 中更美觀地顯示 DataFrame

In [2]:
# MongoDB 連線資訊 (請替換成你的實際資訊)
mongo_uri = 'mongodb://localhost:27017/'  # 預設本機連線
database_name = 'mydatabase'         # 你的資料庫名稱
collection_name = 'reviews'        # 你的 Collection 名稱

try:
    client = MongoClient(mongo_uri)
    db = client[database_name]
    collection = db[collection_name]
    print(f"成功連線到 MongoDB 資料庫 '{database_name}'，Collection: '{collection_name}'")
except Exception as e:
    print(f"連線 MongoDB 失敗: {e}")
    exit()

成功連線到 MongoDB 資料庫 'mydatabase'，Collection: 'reviews'


In [3]:
# print("\n--- 資料庫和 Collection 概覽 ---")

# # 顯示資料庫名稱列表
# database_names = client.list_database_names()
# print("資料庫列表:", database_names)

# # 顯示 Collection 名稱列表 (在當前資料庫中)
# collection_names = db.list_collection_names()
# print("Collection 列表:", collection_names)

# # 顯示 Collection 文件數量
# document_count = collection.count_documents({})
# print(f"\nCollection '{collection_name}' 中的文件總數: {document_count}")

In [4]:
# # 使用 aggregation pipeline 計算不重複的 ProductId 的數量
# pipeline = [
#     {"$group": {"_id": "$ProductId"}},
#     {"$count": "total_products"}
# ]
# result = list(collection.aggregate(pipeline))

# if result:
#     num_unique_products = result[0]["total_products"]
# else:
#     num_unique_products = 0

# print(f"這個 Collection 中共有 {num_unique_products} 種產品。")

## 提取資料庫中所有的 ProductId

已確認非 B 開頭的 ProductId 的商品僅有 8 個，為求方便直接忽略。

In [5]:
# 從 collection 中提取所有獨立的 ProductId
unique_product_ids = collection.distinct("ProductId")

# 將其轉換為 Pandas DataFrame
df_unique_product_ids = pd.DataFrame(unique_product_ids, columns=["ProductId"])

# 篩選出以 "B" 開頭的商品 ID
df_b_products = df_unique_product_ids[df_unique_product_ids['ProductId'].str.startswith('B')]

# 顯示篩選結果
display(df_b_products)

,ProductId
8,B00002N8SM
9,B00002NCJC
10,B00002Z754
11,B00004CI84
12,B00004CXX9
...,...
74253,B009UOFTUI
74254,B009UOFU20
74255,B009UUS05I
74256,B009WSNWC4


## Get Product Metadata for other datasets

https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

In [6]:
import urllib.request
import os

url = "https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/all_categories.txt"
filename = "all_categories.txt"
save_directory = "./"

filepath = os.path.join(save_directory, filename)

if not os.path.exists(filepath):
    try:
        urllib.request.urlretrieve(url, filepath)
        print(f"檔案 '{filename}' 下載完成，儲存於 '{save_directory}'。")
    except Exception as e:
        print(f"下載失敗: {e}")
else:
    print(f"檔案 '{filename}' 已存在於 '{save_directory}'。")

with open(filename, 'r', encoding='utf-8') as f:
    categories = f.readlines() # 將每一行讀取到一個 list 中
    # print(f"檔案 '{filename}' 內容:")
    # for line in categories:
    #     print(line.strip())  # 去掉行尾的換行符號

檔案 'all_categories.txt' 已存在於 './'。


In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm # Optional: for progress bar
import gc # Garbage Collector


# 將我們要查找的本地 ProductId 存儲在一個集合中，以便快速查找
# 這一步在迴圈外執行一次即可
local_product_ids_set = set(df_b_products['ProductId'])
print(f"已載入 {len(local_product_ids_set)} 個以 'B' 開頭的本地 ProductId 供比對。")


# --- 初始化一個集合來收集所有類別中匹配到的 ProductId ---
# 使用集合可以自動處理重複的 ID (如果一個商品在多個類別的 dataset 中都出現)
all_matched_product_ids = set()

# --- 迭代處理每個類別 ---
for category in categories:
    category = category.strip()
    print(f"\n--- 開始處理類別: {category} ---")
    
    dataset_stream = None # 初始化變數以確保 finally 子句可以存取
    
    try:
        # 1. 使用串流模式載入資料集
        print(f"以串流模式載入資料集: raw_meta_{category}...")
        dataset_stream = load_dataset(
            "McAuley-Lab/Amazon-Reviews-2023",
            "raw_meta_" + category,
            split="full",
            streaming=True,
            trust_remote_code=True
        )

        # 2. 迭代串流資料集，提取 parent_asin 並查找匹配項
        print("迭代串流資料集以查找匹配的 parent_asin...")
        
        matched_in_this_category = set() # 記錄此類別找到的匹配 ID
        processed_count = 0

        # 使用 tqdm 顯示進度條 (可選)
        for record in tqdm(dataset_stream, desc=f"處理 {category}"):
            processed_count += 1
            # 確保 record 中有 'parent_asin' 鍵且值不是 None
            if record and 'parent_asin' in record and record['parent_asin'] is not None:
                dataset_asin = str(record['parent_asin']) # 確保是字串

                # 如果資料集的 ASIN 存在於我們的本地 ID 集合中
                if dataset_asin in local_product_ids_set:
                    # 將這個匹配到的 ID 添加到 *總的* 匹配 ID 集合中
                    all_matched_product_ids.add(dataset_asin)
                    # 也添加到此類別的集合中，用於報告統計信息
                    matched_in_this_category.add(dataset_asin)

        print(f"類別 '{category}' 處理完成。此類別找到 {len(matched_in_this_category)} 個匹配的 ProductId。")
        print(f"目前累積找到 {len(all_matched_product_ids)} 個唯一的匹配 ProductId。")

    except Exception as e:
        print(f"處理類別 '{category}' 時發生錯誤: {e}")
        # 決定是否繼續處理下一個類別
        continue # 繼續下一個類別
        # break    # 中止迴圈
        
    finally:
        # --- 清理資源 ---
        # 關閉可能打開的文件句柄或連接（雖然 datasets 通常會自己管理，但顯式刪除有助於 RAM）
        del dataset_stream 
        gc.collect() # 嘗試觸發垃圾回收以釋放記憶體
        print(f"已清理類別 '{category}' 的資源。")


# --- 所有類別處理完成後 ---
print("\n--- 所有類別處理完成 ---")

# 3. 根據收集到的所有匹配 ID，從原始 df_b_products 中篩選最終結果
if all_matched_product_ids:
    print(f"總共找到 {len(all_matched_product_ids)} 個唯一的匹配 ProductId。")
    print("正在從本地 DataFrame 中篩選最終匹配的商品...")
    
    # 使用 isin() 進行高效篩選
    final_matched_df = df_b_products[df_b_products['ProductId'].isin(all_matched_product_ids)].copy()
    # 再次使用 .copy() 確保我们得到的是獨立的 DataFrame

    # 4. 將整合後的匹配結果儲存為單一 CSV 文件
    output_filename = "all_matched_products.csv"
    print(f"將所有匹配結果 ({len(final_matched_df)} 筆) 儲存到: {output_filename}")
    
    try:
        final_matched_df.to_csv(output_filename, index=False, encoding='utf-8')
        print("CSV 文件儲存成功。")
    except Exception as e:
        print(f"儲存 CSV 文件時發生錯誤: {e}")

else:
    print("在所有處理的類別中，沒有找到任何匹配的商品。")

print("\n--- 腳本執行完畢 ---")

/home/hank/miniconda3/envs/mongo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


已載入 74250 個以 'B' 開頭的本地 ProductId 供比對。

--- 開始處理類別: All_Beauty ---
以串流模式載入資料集: raw_meta_All_Beauty...
迭代串流資料集以查找匹配的 parent_asin...


處理 All_Beauty: 112590it [00:25, 4431.51it/s]


類別 'All_Beauty' 處理完成。此類別找到 3 個匹配的 ProductId。
目前累積找到 3 個唯一的匹配 ProductId。
已清理類別 'All_Beauty' 的資源。

--- 開始處理類別: Toys_and_Games ---
以串流模式載入資料集: raw_meta_Toys_and_Games...
迭代串流資料集以查找匹配的 parent_asin...


處理 Toys_and_Games: 626672it [03:36, 3896.56it/s]'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 39f0bfa0-14e1-44e4-b833-9d44f9a0c5b7)')' thrown while requesting GET https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/2b6d039ed471f2ba5fd2acb718bf33b0a7e5598e/raw/meta_categories/meta_Toys_and_Games.jsonl
Retrying in 1s [Retry 1/5].
處理 Toys_and_Games: 626672it [03:52, 3896.56it/s]'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 1defe6b5-2cbf-414f-a663-72cdecf15c78)')' thrown while requesting GET https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/2b6d039ed471f2ba5fd2acb718bf33b0a7e5598e/raw/meta_categories/meta_Toys_and_Games.jsonl
Retrying in 2s [Retry 2/5].
處理 Toys_and_Games: 890874it [05:32, 2677.31it/s]


...

處理 Office_Products: 710503it [05:59, 1973.65it/s]


類別 'Office_Products' 處理完成。此類別找到 8 個匹配的 ProductId。
目前累積找到 959 個唯一的匹配 ProductId。
已清理類別 'Office_Products' 的資源。

--- 開始處理類別: Digital_Music ---
以串流模式載入資料集: raw_meta_Digital_Music...
迭代串流資料集以查找匹配的 parent_asin...


處理 Digital_Music: 70537it [00:13, 5350.41it/s]


類別 'Digital_Music' 處理完成。此類別找到 1 個匹配的 ProductId。
目前累積找到 960 個唯一的匹配 ProductId。
已清理類別 'Digital_Music' 的資源。

--- 開始處理類別: Grocery_and_Gourmet_Food ---
以串流模式載入資料集: raw_meta_Grocery_and_Gourmet_Food...
迭代串流資料集以查找匹配的 parent_asin...


處理 Grocery_and_Gourmet_Food: 82836it [00:35, 2900.91it/s]